In [96]:
#Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [97]:
#Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}

browser = Browser('chrome', **executable_path, headless=False)

In [98]:
#Visit the mars nasa news site
url = 'https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html'

browser.visit(url)

#Optional delay for loading the page
browser.is_element_present_by_css('table.table', wait_time=1)

True

In [99]:
#Set up the HTML parser
html = browser.html

data_soup = soup(html, 'html.parser')

In [100]:
#Get each table row element
table_rows = data_soup.find_all('tr', class_='data-row'.split())

In [101]:
#Create lists to hold each column data
id_ = []
terrestrial_date = []
sol = []
ls = []
month = []
min_temp = []
pressure = []

In [102]:
#Iterate through each table row and add data to lists
for row in table_rows:
    #Turn each table row into a list with each column as an element
    row_data = row.find_all('td')
    
    #Add each column data to the appropriate list
    #When I try using .astype here, I get an error that says "AttributeError: 'NoneType' object has no attribute 'astype'" even though .info below shows that there is no null data
    id_.append(row_data[0].text)
    terrestrial_date.append(row_data[1].text)
    sol.append(row_data[2].text)
    ls.append(row_data[3].text)
    month.append(row_data[4].text)
    min_temp.append(row_data[5].text)
    pressure.append(row_data[6].text)

In [103]:
#Create dataframe from lists
mars_temp_df = pd.DataFrame({'id': id_, 'terrestrial_date': terrestrial_date, 'sol': sol, 'ls': ls, 'month': month, 'min_temp': min_temp, 'pressure': pressure})

mars_temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1867 entries, 0 to 1866
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                1867 non-null   object
 1   terrestrial_date  1867 non-null   object
 2   sol               1867 non-null   object
 3   ls                1867 non-null   object
 4   month             1867 non-null   object
 5   min_temp          1867 non-null   object
 6   pressure          1867 non-null   object
dtypes: object(7)
memory usage: 102.2+ KB


In [105]:
#Convert columns to appropriate data types
mars_temp_df['id'] = mars_temp_df['id'].astype(int)
mars_temp_df['terrestrial_date'] = pd.to_datetime(mars_temp_df['terrestrial_date'])
mars_temp_df['sol'] = mars_temp_df['sol'].astype(int)
mars_temp_df['ls'] = mars_temp_df['ls'].astype(int)
mars_temp_df['month'] = mars_temp_df['month'].astype(int)
mars_temp_df['min_temp'] = mars_temp_df['min_temp'].astype(float)
mars_temp_df['pressure'] = mars_temp_df['pressure'].astype(float)